In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Albany,US,2022-01-31 13:42:44,42.6001,-73.9662,18.30,72,21,1.74
1,1,Katobu,ID,2022-01-31 13:47:20,-4.9408,122.5276,76.87,81,95,1.74
2,2,Ribeira Grande,PT,2022-01-31 13:47:21,38.5167,-28.7000,63.00,59,20,21.85
3,3,Hobart,AU,2022-01-31 13:46:57,-42.8794,147.3294,67.12,87,40,4.61
4,4,Andros Town,BS,2022-01-31 13:47:21,24.7000,-77.7667,66.16,52,96,14.12


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
# Heatmap of temperature
# 1. Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# 2. Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# 3. Assign the figure variable.
fig = gmaps.figure( center=(30.0, 31.0), zoom_level=1.5)
# 4. Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=[max(temp,0) for temp in max_temp], 
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)
# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Heatmap of temperature
# 1. Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# 2. Get the maximum temperature.
humidity = city_data_df["Humidity"]
# 3. Assign the figure variable.
fig = gmaps.figure( center=(30.0, 31.0), zoom_level=1.5)
# 4. Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)
# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of temperature
# 1. Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# 2. Get the maximum temperature.
clouds = city_data_df["Cloudiness"]
# 3. Assign the figure variable.
fig = gmaps.figure( center=(30.0, 31.0), zoom_level=1.5)
# 4. Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=clouds, 
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)
# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of temperature
# 1. Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# 2. Get the maximum temperature.
wind = city_data_df["Wind Speed"]
# 3. Assign the figure variable.
fig = gmaps.figure( center=(30.0, 31.0), zoom_level=1.5)
# 4. Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=wind, 
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)
# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [30]:
preferred_cities_df  = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                        (city_data_df["Max Temp"] <= max_temp)]
#.dropna() ??????
hotel_df = preferred_cities_df[["City_ID", "City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()


,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
1,1,Katobu,ID,76.87,-4.9408,122.5276,
6,6,Bengkulu,ID,78.28,-3.8004,102.2655,
9,9,Vaini,TO,79.57,-21.2000,-175.2000,
10,10,Namibe,AO,77.68,-15.1961,12.1522,
12,12,Santa Cruz,BO,82.45,-17.8000,-63.1667,


In [31]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [32]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    #Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
    
    
    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
hotel_df

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
1,1,Katobu,ID,76.87,-4.9408,122.5276,La Ode Husein
6,6,Bengkulu,ID,78.28,-3.8004,102.2655,Grage Hotel Bengkulu
9,9,Vaini,TO,79.57,-21.2000,-175.2000,Keleti Beach Resort
10,10,Namibe,AO,77.68,-15.1961,12.1522,Hotel Chik Chik Namibe
12,12,Santa Cruz,BO,82.45,-17.8000,-63.1667,Aviador Hotel Boutique
...,...,...,...,...,...,...,...
524,524,Riachao,BR,84.29,-7.3619,-46.6172,Pousada Nordestina
526,526,Kupang,ID,82.98,-10.1667,123.5833,Hotel On The Rock by Prasanthi
528,528,Pandan,PH,77.81,14.0453,124.1694,Bonifacio Lodge
531,531,Hervey Bay,AU,78.04,-25.2986,152.8535,Shelly Bay Resort


In [37]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [38]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))